## Tratamento do PPA por regionalização

Nesse notebook são tratados os dados do Plano Plurianual por regionalização (2022 - 2025) da Prefeitura Municipal de São Paulo

In [1]:
import pandas as pd
from utils.load_csv import load_csv
from utils.save_csv import save_csv

In [2]:
df_ppa_reg = load_csv("ppa_reg.csv")

#### Criação "id"

Cria variável "id" concatenando variáveis de código da base de dados para "chave" do merge

In [3]:
df_ppa_reg["id"] = df_ppa_reg[["codigo orgao",
                               "codigo unidade",
                               "codigo programa",
                               "codigo acao"]].apply(lambda x: "_".join(x.astype(str)), axis = 1)

In [4]:
cols = ["id"] + [col for col in df_ppa_reg.columns if col != "id"]

df_ppa_reg = df_ppa_reg[cols]

#### Transformação das variáveis de planejamento de recurso

Transforma as variáveis de planejamento de resurso da base de dados de string para float

In [5]:
cols_plan_recurso = ["valor 2022",
                     "valor 2023",
                     "valor 2024",
                     "valor 2025"]

In [6]:
for col in cols_plan_recurso:
    df_ppa_reg[col] = (df_ppa_reg[col]
                         .str.replace(".", "")
                         .str.replace(",", ".")
                         .astype(float))

#### Filtro regionalizável

Filtra a base de dados para conter apenas despesas regionalizáveis

In [7]:
supra_subprefs = ["Supra Subprefeitura",
                  "Supra Subprefeitura Leste",
                  "Supra Subprefeitura Oeste",
                  "Supra Subprefeitura Norte",
                  "Supra Subprefeitura Sul",
                  "Supra Subprefeitura Centro"]

In [8]:
df_ppa_reg = df_ppa_reg[~df_ppa_reg["descricao prefeitura regional"].isin(supra_subprefs)]

#### Normalização das subprefeituras

Trata os nomes das subprefreituras na base de dados para estar igual aos nomes do shapefile das subprefeituras

In [9]:
mapper = {"Subprefeitura Sé": "SE",
          "Subprefeitura Vila Mariana": "VILA MARIANA",
          "Subprefeitura Santana/Tucuruvi": "SANTANA-TUCURUVI",
          "Subprefeitura Campo Limpo": "CAMPO LIMPO",
          "Subprefeitura Itaim Paulista": "ITAIM PAULISTA",
          "Subprefeitura Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
          "Subprefeitura Sapopemba": "SAPOPEMBA",
          "Subprefeitura Capela do Socorro": "CAPELA DO SOCORRO",
          "Subprefeitura Lapa": "LAPA",
          "Subprefeitura Jaçanã/Tremembé": "JACANA-TREMEMBE",
          "Subprefeitura Penha": "PENHA",
          "Subprefeitura São Miguel Paulista": "SAO MIGUEL",
          "Subprefeitura Perus/Anhanguera": "PERUS",
          "Subprefeitura Butantã": "BUTANTA",
          "Subprefeitura de Guaianases": "GUAIANASES",
          "Subprefeitura São Mateus": "SAO MATEUS",
          "Subprefeitura Cidade Ademar": "CIDADE ADEMAR",
          "Subprefeitura de Vila Prudente": "VILA PRUDENTE",
          "Subprefeitura Ipiranga": "IPIRANGA",
          "Subprefeitura Pinheiros": "PINHEIROS",
          "Subprefeitura Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
          "Subprefeitura Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
          "Subprefeitura M'Boi Mirim": "M BOI MIRIM",
          "Subprefeitura Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
          "Subprefeitura Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
          "Subprefeitura Santo Amaro": "SANTO AMARO",
          "Subprefeitura Mooca": "MOOCA",
          "Subprefeitura Ermelino Matarazzo": "ERMELINO MATARAZZO",
          "Subprefeitura Cidade Tiradentes": "CIDADE TIRADENTES",
          "Subprefeitura Itaquera": "ITAQUERA",
          "Subprefeitura Jabaquara": "JABAQUARA",
          "Subprefeitura Parelheiros": "PARELHEIROS"}

In [10]:
df_ppa_reg.loc[:, "descricao prefeitura regional"] = df_ppa_reg["descricao prefeitura regional"].map(mapper)

#### Salvar

In [11]:
pd.set_option("display.max_columns", None)

df_ppa_reg.sample(10)

,id,codigo orgao,descricao orgao,codigo unidade,descricao unidade,codigo programa,descricao programa,codigo acao,descricao acao,codigo da,descricao da,codigo meta,descricao meta,codigo regiao,descricao regiao,codigo prefeitura regional,descricao prefeitura regional,codigo distrito,descricao distrito,codigo sequencial da,codigo produto,descricao produto,descricao unidade medida,quantidade 2022,valor 2022,quantidade 2023,valor 2023,quantidade 2024,valor 2024,quantidade 2025,valor 2025
6843,34_10_3013_4322,34,Secretaria Municipal de Direitos Humanos e Cid...,10,Gabinete do Secretário,3013,"Promoção da Segurança Urbana, Prevenção e Prot...",4322,"Políticas, Programas e Ações para Povos Indígenas",777.02.30.00.001,Subprefeitura São Mateus,777,Despesa Regionalizável,2,Leste,30,SAO MATEUS,0,Supra-Distrital,1,2886,"Ações (campanhas publicitárias, lives, eventos...",ações,"0,00",5000.0,"1,00",5489.0,1,5544.0,0,5879.0
7278,54_10_3024_2100,54,Subprefeitura Santo Amaro,10,Administração da Subprefeitura,3024,Suporte Administrativo,2100,Administração da Unidade,777.04.14.00.001,Subprefeitura Santo Amaro,777,Despesa Regionalizável,4,Sul,14,SANTO AMARO,0,Supra-Distrital,1,2977,Operação/Serviço mantido,unida,"1,00",16963859.0,"1,00",17881686.0,1,18649716.0,1,19756183.0
2182,16_11_3025_4362,16,Secretaria Municipal de Educação,11,Diretoria Regional de Educação Ipiranga,3025,Proteção e Desenvolvimento Integral na Primeir...,4362,Manutenção e Operação de Escolas Municipais de...,777.04.12.00.001,Subprefeitura Vila Mariana,777,Despesa Regionalizável,4,Sul,12,VILA MARIANA,0,Supra-Distrital,1,2633,EMEIs em funcionamento,unida,"0,00",723147.0,"0,00",970642.0,0,975949.0,0,1018692.0
624,12_10_3022_2387,12,Secretaria Municipal das Subprefeituras,10,Gabinete do Secretário,3022,Requalificação e Promoção da Ocupação dos Espa...,2387,Ações de Fiscalização do Comércio Ilegal,777.04.19.00.001,Subprefeitura Capela do Socorro,777,Despesa Regionalizável,4,Sul,19,CAPELA DO SOCORRO,0,Supra-Distrital,1,2811,Termos de Ajustamento de Conduta (TAC) decorre...,unida,"0,00",2403337.0,"0,00",1799514.0,0,1946209.0,0,2114409.0
4603,25_10_3001_6365,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,6365,Programa de Articulação Criativa,777.04.20.00.001,Subprefeitura Parelheiros,777,Despesa Regionalizável,4,Sul,20,PARELHEIROS,0,Supra-Distrital,1,2909,Participantes do Programa Articulação Criativa...,parti,"2,00",16200.0,"2,00",17010.0,2,17860.0,2,18753.0
7779,75_10_3005_1702,75,Fundo Municipal de Parques,10,Fundo Municipal de Parques,3005,Promoção da Sustentabilidade Ambiental,1702,Construção e Implantação de Parques Urbanos e ...,777.02.29.00.001,Subprefeitura de Vila Prudente,777,Despesa Regionalizável,2,Leste,29,VILA PRUDENTE,0,Supra-Distrital,1,2766,Novos parques com obras iniciadas,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
5727,25_10_3001_6702,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,6702,Políticas de Audiovisual,777.02.22.00.001,Subprefeitura Ermelino Matarazzo,777,Despesa Regionalizável,2,Leste,22,ERMELINO MATARAZZO,0,Supra-Distrital,1,2904,Atividades culturais realizadas,ações,"0,00",801550.0,"1,00",1512090.0,1,582153.0,1,611259.0
404,12_10_3016_4301,12,Secretaria Municipal das Subprefeituras,10,Gabinete do Secretário,3016,Nutrição e Promoção da Segurança Alimentar,4301,Ações Municipais de Abastecimento,777.02.25.00.001,Subprefeitura Mooca,777,Despesa Regionalizável,2,Leste,25,MOOCA,0,Supra-Distrital,1,2874,Ações sanitárias,ações,"0,00",34396.0,"0,00",25753.0,0,27852.0,0,30259.0
6443,29_30_3022_3662,29,Secretaria Municipal de Urbanismo e Licenciamento,30,Operação Urbana Consorciada Água Espraiada,3022,Requalificação e Promoção da Ocupação dos Espa...,3662,Apoio e Suporte Técnico para o Desenvolvimento...,777.04.17.00.001,Subprefeitura Campo Limpo,777,Despesa Regionalizável,4,Sul,17,CAMPO LIMPO,0,Supra-Distrital,1,2978,Projeto concluído,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
4175,25_10_3001_5406,2

In [12]:
save_csv(df_ppa_reg, "ppa_reg.csv")

Base salva em data\ppa_reg.csv
